In [1]:
! pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.0 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize

In [6]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
# Run 50 trials to find the best hyperparameters
study.optimize(objective, n_trials=50)

[I 2024-10-14 03:17:42,599] A new study created in memory with name: no-name-30d683db-4155-49ae-8763-d6d7794259d5
[I 2024-10-14 03:17:43,672] Trial 0 finished with value: 0.7765363128491621 and parameters: {'n_estimators': 193, 'max_depth': 17}. Best is trial 0 with value: 0.7765363128491621.
[I 2024-10-14 03:17:44,436] Trial 1 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 138, 'max_depth': 15}. Best is trial 0 with value: 0.7765363128491621.
[I 2024-10-14 03:17:44,843] Trial 2 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 74, 'max_depth': 9}. Best is trial 0 with value: 0.7765363128491621.
[I 2024-10-14 03:17:45,677] Trial 3 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 151, 'max_depth': 11}. Best is trial 0 with value: 0.7765363128491621.
[I 2024-10-14 03:17:46,534] Trial 4 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 155, 'max_depth': 18}. Best is trial 0 with value: 0.776536

In [7]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 186, 'max_depth': 16}


### RandomSampler

In [ ]:
# study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())
# study.optimize(objective, n_trials=50)

### GridSampler

In [ ]:
# search_space = {
#     'n_estimators': [50, 100, 150, 200],
#     'max_depth': [5, 10, 15, 20]
# }
# study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
# study.optimize(objective)

## Optuna Visualizations

In [8]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [9]:
# Optimization History
plot_optimization_history(study).show()

In [10]:
# Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [11]:
# Slice Plot
plot_slice(study).show()

In [12]:
# Contour Plot
plot_contour(study).show()

In [13]:
# Hyperparameter Importance
plot_param_importances(study).show()

Ref: https://optuna.org/ <br>
Thanks to : Campus X